# 3D UNet


In [ ]:
# hide
import sys
sys.path.append("..")
from nbdev.showdoc import *

In [ ]:
# default_exp models.unet
# export 
from fastai.basics import *
from fastai.layers import *
import torchvision, torch
from warnings import warn
from faimed3d.models.resnet import *

## Building Blocks


### Convolutional BlocKs
#### DoubleConv
Standard double Convulutional-Block for UNet adapted for 3D.


In [ ]:
# export
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


### Upscaling Block

In [ ]:
# export
class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels, out_channels, trilinear=True):
        super().__init__()

        # if trilinear, use the normal convolutions to reduce the number of channels
        if trilinear:
            self.up = nn.Sequential(
                nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True),
                DoubleConv(in_channels, in_channels // 2)
            )
            self.conv = DoubleConv(in_channels, out_channels, in_channels // 2)
        else:
            self.up = nn.ConvTranspose3d(in_channels , in_channels // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels, out_channels)


    def forward(self, x1, x2):
        
        x1 = self.up(x1)
        # input is CHW
        diffZ = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        diffX = x2.size()[4] - x1.size()[4]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2, 
                        diffZ // 2, diffZ - diffZ // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

### Out Layers

In [ ]:
# export
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1)
            
    def forward(self, x):
        return self.conv(x)

## Models

### Base Classes

UNet architecture as initially described by [Ronneberger et al](https://arxiv.org/abs/1505.04597).

In [ ]:
# export
class AbstractUNet3D(nn.Module):
    "Abstract base class of UNet, should be subclassed by all UNets"
    def __init__(self):
        super(AbstractUNet3D, self).__init__()
    
    def forward(self, x):
        x1, x2, x3, x4, x5 = self.encoder(x)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

### 3D UResNet

3D version of the original UNet with ResNet3D Encoder

In [ ]:
# export
class UResNet3D(AbstractUNet3D):
    def __init__(self, backbone, n_channels, n_classes, trilinear=False):
        super(UResNet3D, self).__init__()
        self.encoder=build_backbone(backbone, n_channels=n_channels, output_stride=16, BatchNorm=nn.BatchNorm3d)
        self.up1 = Up(2048, 1024, trilinear)
        self.up2 = Up(1024, 512, trilinear)
        self.up3 = Up(512, 256, trilinear)
        self.up4 = Up(256, 128, trilinear)
        self.outc = OutConv(128, n_classes)

In [ ]:
UResNet3D(resnet152_3d, 4, 3, trilinear=True)(torch.randn(10, 4, 10, 80, 80)).size()

torch.Size([10, 3, 3, 20, 20])

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 01_basics.ipynb.
Converted 02_transforms.ipynb.
Converted 03_datablock.ipynb.
Converted 04_datasets.ipynb.
Converted 05a_models.modules.ipynb.
Converted 05b_models.deeplabv3.ipynb.
Converted 05b_models.resnet.ipynb.
Converted 05c_models.unet.ipynb.
Converted 05d_models.losses.ipynb.
Converted 06_callback.ipynb.
Converted 99_tools.ipynb.
Converted index.ipynb.
